In [48]:
# Cell 1: Install necessary libraries (run this if not installed)
!pip install mysql-connector-python pymongo pandas

# Import mysql.connector and create a MySQL connection without specifying a database
import mysql.connector

# Connect to MySQL server (without specifying a database)
mysql_conn = mysql.connector.connect(
    host='localhost',     # Replace with your MySQL host
    user='root', # Replace with your MySQL username
    password='Root@1234' # Replace with your MySQL password
)
mysql_cursor = mysql_conn.cursor()

# Create the database if it doesn't already exist
mysql_cursor.execute("CREATE DATABASE IF NOT EXISTS Stock_DB")
print("MySQL database created successfully.")

# Now, reconnect to use the new database
mysql_conn.database = 'Stock_DB'



Defaulting to user installation because normal site-packages is not writeable
MySQL database created successfully.


In [35]:
# Cell 2: Import MongoClient and create a MongoDB connection
from pymongo import MongoClient

# Connect to MongoDB server
mongo_client = MongoClient('mongodb://localhost:27017/')  # Replace with your MongoDB URI if needed

# Define the database
mongo_db = mongo_client['Stock_DB']  # Replace with your MongoDB database name
print("MongoDB database created successfully.")


MongoDB database created successfully.


In [36]:
# Cell 3: Create tables in MySQL
# Create table for stock tweets
mysql_cursor.execute("""
    CREATE TABLE IF NOT EXISTS stock_tweets (
        id INT PRIMARY KEY,
        date DATE,
        ticker VARCHAR(10),
        tweet TEXT
    );
""")

# Create single table for all stock prices
mysql_cursor.execute("""
    CREATE TABLE IF NOT EXISTS stock_prices (
        date DATE,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        adj_close FLOAT,
        volume BIGINT,
        ticker VARCHAR(10)
    );
""")

# Commit changes to create tables
mysql_conn.commit()
print("MySQL tables created successfully.")


MySQL tables created successfully.


In [37]:
# Cell 4: Import libraries for data manipulation
import pandas as pd
import glob
import os

# Define paths to CSV files
tweets_file = "stocktweet/stocktweet.csv"  # Replace with the path to your stock tweets CSV
stock_files_path = "stockprice/*.csv"  # Replace with the path to your stock prices folder

# Load stock tweets data
tweets_df = pd.read_csv(tweets_file)

# Load each stock price file and add ticker column
stock_dataframes = {}
for file in glob.glob(stock_files_path):
    ticker = os.path.basename(file).replace(".csv", "")
    df = pd.read_csv(file)
    df['ticker'] = ticker  # Add ticker column to identify the stock
    stock_dataframes[ticker] = df

print("Data loaded successfully.")


Data loaded successfully.


In [38]:
# Cell 5: Data cleaning
# Check for missing values in tweets data and fill if necessary
tweets_df.fillna("", inplace=True)

# Convert 'Date' columns to datetime format
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')
for ticker, df in stock_dataframes.items():
    df['date'] = pd.to_datetime(df['Date'], errors='coerce')
    df.dropna(subset=['date'], inplace=True)  # Drop rows with invalid dates

print("Data cleaned successfully.")


Data cleaned successfully.


In [39]:
# Cell 6: Insert data into MySQL
# Insert tweets data into MySQL
for _, row in tweets_df.iterrows():
    mysql_cursor.execute("""
        INSERT INTO stock_tweets (id, date, ticker, tweet)
        VALUES (%s, %s, %s, %s)
    """, (
        row['id'], row['date'], row['ticker'], row['tweet']
    ))

# Insert stock prices data into MySQL
for ticker, df in stock_dataframes.items():
    for _, row in df.iterrows():
        mysql_cursor.execute("""
            INSERT INTO stock_prices (date, open, high, low, close, adj_close, volume, ticker)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['date'], row['Open'], row['High'], row['Low'], row['Close'],
            row['Adj Close'], row['Volume'], row['ticker']
        ))

# Commit changes
mysql_conn.commit()
print("Data inserted into MySQL successfully.")


IntegrityError: 1062 (23000): Duplicate entry '100001' for key 'stock_tweets.PRIMARY'

In [46]:
# Cell 7: Insert data into MongoDB

# MongoDB collections for tweets and stock prices
tweets_collection = mongo_db['stock_tweets']
prices_collection = mongo_db['stock_prices']

# Ensure datetime column in tweets data is correctly parsed and NaT values are handled
if 'date' in tweets_df.columns:
    # Convert invalid datetime entries to None
    def convert_to_datetime(value):
        try:
            # Try converting to datetime
            return pd.to_datetime(value)
        except (ValueError, TypeError):
            # If conversion fails, return None
            return None
    print('inside date time if condtion')
    tweets_df['datetime'] = tweets_df['date'].apply(convert_to_datetime)
    print("Processed tweets_df datetime values:", tweets_df['datetime'].head())  # Debugging output
print(tweets_df)
# Insert tweets data into MongoDB with error handling
try:
    tweets_collection.insert_many(tweets_df.to_dict(orient='records'))
    print(f"{len(tweets_df)} tweets inserted into MongoDB.")
except Exception as e:
    print(f"Error inserting tweets into MongoDB: {e}")

# Insert stock prices data into MongoDB
for ticker, df in stock_dataframes.items():
    # Ensure datetime column in stock data is correctly parsed and NaT values are handled
    if 'datetime' in df.columns:
        # Convert invalid datetime entries to NaT, then replace NaT with None for MongoDB
        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce').apply(lambda x: None if pd.isna(x) else x)
        print(f"Processed datetime values for {ticker}:", df['datetime'].head())  # Debugging output

    try:
        records = df.to_dict(orient='records')
        prices_collection.insert_many(records)
        print(f"{len(records)} records for {ticker} inserted into MongoDB.")
    except Exception as e:
        print(f"Error inserting stock prices for {ticker} into MongoDB: {e}")

print("Data insertion completed.")


inside date time if condtion
Processed tweets_df datetime values: 0   2020-01-01
1   2020-01-01
2   2020-01-01
3   2020-01-01
4   2020-01-01
Name: datetime, dtype: datetime64[ns]
          id       date ticker  \
0     100001 2020-01-01   AMZN   
1     100002 2020-01-01   TSLA   
2     100003 2020-01-01   AAPL   
3     100004 2020-01-01   TSLA   
4     100005 2020-01-01   TSLA   
...      ...        ...    ...   
9995  109996        NaT   ABNB   
9996  109997        NaT   TSLA   
9997  109998        NaT   BABA   
9998  109999        NaT    CCL   
9999  110000        NaT   TSLA   

                                                  tweet   datetime  
0          $AMZN Dow futures up by 100 points already 🥳 2020-01-01  
1     $TSLA Daddy's drinkin' eArly tonight! Here's t... 2020-01-01  
2     $AAPL We’ll been riding since last December fr... 2020-01-01  
3               $TSLA happy new year, 2020, everyone🍷🎉🙏 2020-01-01  
4     $TSLA haha just a collection of greats..."Mars... 2020-01-01 

In [47]:
# Cell 8: Close connections
mysql_cursor.close()
mysql_conn.close()
mongo_client.close()

print("Connections closed successfully.")


Connections closed successfully.
